In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from methods import data_handler
from tqdm import tqdm

from torchvision import datasets
import os
from itertools import chain


Bad key "text.kerning_factor" on line 4 in
/Users/julianvonklitzing/opt/anaconda3/envs/xai_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/julianvonklitzing/opt/anaconda3/envs/xai_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
questionnaires = data_handler.get_questionaires("data2/questionaires_shuffled.pickle")
labels = data_handler.get_labels()

Using downloaded and verified file: ./data/imagenet_class_index.json


In [33]:
def get_label_from_img_idx(img_idx, testset_path, labels):    
    img_folder = datasets.ImageFolder(root=testset_path)
    img_path = img_folder.imgs[img_idx][0]
    img_name = img_path.split(os.sep)[-1]
    # extract correct class
    class_idx_true_str = img_path.split(os.sep)[-2]
    img_label_true = labels[class_idx_true_str][1]
    return img_label_true

In [34]:
questionnaires_2 = []
for questionnaire in tqdm(questionnaires):
    questionnaire_2 = []
    for question in questionnaire:
        label = get_label_from_img_idx(question[0], "data2/imagenetv2-matched-frequency-format-val", labels)
        question_labled = (label, ) + question        
        questionnaire_2.append(question_labled)
    questionnaires_2.append(questionnaire_2) 

100%|███████████████████████████████████████████| 12/12 [00:36<00:00,  3.03s/it]


In [35]:
df = pd.read_excel("data2/data_tu-helpfulness-of-xai_2022-06-21_17-15.xlsx")

In [29]:
qn = 1
for idx, qu in enumerate(questionnaires_2[qn]):
    print(df[df.columns[6 + qn*24:6 + qn*24 + 24]].iloc[0][idx], "\t", qu[0])

IndexError: list index out of range

In [ ]:
"""
0: passt nicht
1: passt
2-7: passt nicht
8:passt
9-11: passt nicht

Warum passen 1 und 8??
"""

## Load mapping

In [36]:
codes_list = []
for i in range(1, 12+1):
    codes = pd.read_csv(f"questionaires_shuffle_order/questionaire_{i}.txt", sep=";", names=[0, 1])[1]
    codes = codes.str.extract("(\w\d{3})")
    codes = list(codes[0])
    codes_list.append(codes)

In [43]:
df_quest_meta = pd.DataFrame(list(chain(*questionnaires_2)))
df_quest_meta[5] = list(chain(*codes_list))
df_quest_meta.columns = ["label", "img_idx", "model", "method", "success", "quest_code"]

In [44]:
df_quest_meta

,label,img_idx,model,method,success,quest_code
0,custard_apple,9521,vgg,LRP,True,B108
1,Boston_bull,1073,vgg,SHAP,True,B111
2,toilet_tissue,9992,alex,IntegratedGradients,False,B118
3,gas_pump,5253,vgg,ConfidenceScores,False,B123
4,mailbag,5976,vgg,IntegratedGradients,False,B124
...,...,...,...,...,...,...
283,custard_apple,9521,vgg,gradCAM,True,C307
284,coral_reef,9710,alex,gradCAM,True,C302
285,black_grouse,7798,alex,LRP,False,C319
286,steel_arch_bridge,8034,alex,SHAP,True,C317


In [25]:
df.T.index[:50]

Index(['CASE', 'SERIAL', 'REF', 'QUESTNNR', 'MODE', 'STARTED', 'B101', 'B102',
       'B103', 'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B110', 'B111',
       'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 'B118', 'B119', 'B120',
       'B121', 'B122', 'B123', 'B124', 'B210', 'B206', 'B221', 'B217', 'B212',
       'B218', 'B203', 'B213', 'B219', 'B223', 'B208', 'B220', 'B207', 'B209',
       'B205', 'B204', 'B215', 'B201', 'B222', 'B224'],
      dtype='object')

In [26]:
df

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,B101,B102,B103,B104,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,Interview-Nummer (fortlaufend),Seriennummer (sofern verwendet),Referenz (sofern im Link angegeben),"Fragebogen, der im Interview verwendet wurde",Interview-Modus,Zeitpunkt zu dem das Interview begonnen hat (E...,q1coral,q2boathouse,q3watertower,q4goose,...,Versandzeitpunkt der Einladungsmail (nur für n...,Zeitpunkt als der Datensatz das letzte mal geä...,Wurde die Befragung abgeschlossen (letzte Seit...,Hat der Teilnehmer den Fragebogen nur angesehe...,"Seite, die der Teilnehmer zuletzt bearbeitet hat","Letzte Seite, die im Fragebogen bearbeitet wurde",Anteil fehlender Antworten in Prozent,Anteil fehlender Antworten (gewichtet nach Rel...,Maluspunkte für schnelles Ausfüllen,Maluspunkte für schnelles Ausfüllen
1,579,CASE000578,578,bogen7,interview,2022-06-13 15:44:02,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 15:46:48,1,0,5,5,4,1,NaN,100
2,581,CASE000580,580,bogen12,interview,2022-06-13 15:44:06,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 15:46:27,1,0,5,5,3,1,2.28,83
3,587,CASE000586,586,bogen11,interview,2022-06-13 15:44:26,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 15:48:03,1,0,5,5,4,1,1.61,32
4,589,CASE000588,588,bogen2,interview,2022-06-13 15:45:57,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 15:48:13,1,0,5,5,7,1,1.92,209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,1558,CASE001557,1557,bogen3,interview,2022-06-21 15:51:28,NaN,NaN,NaN,NaN,...,NaN,2022-06-21 15:58:00,1,0,5,5,0,0,0.78,5
118,1560,CASE001559,1559,bogen12,interview,2022-06-21 15:58:58,NaN,NaN,NaN,NaN,...,NaN,2022-06-21 16:04:34,1,0,5,5,4,1,1.33,31
119,1562,CASE001561,1561,bogen9,interview,2022-06-21 16:02:49,NaN,NaN,NaN,NaN,...,NaN,2022-06-21 16:07:07,1,0,5,5,4,1,NaN,100
120,1564,CASE001563,1563,bogen6,interview,2022-06-21 16:10:55,NaN,NaN,NaN,NaN,...,NaN,2022-06-21 16:25:23,1,0,5,5,3,1,NaN,100
